In [1]:
import pandas as pd
import mysql.connector as mc

In [2]:
db = mc.connect(
  host='172.18.55.6',
  user='comandato',
  passwd='comandato123',
)

In [3]:
Anio='2019'
Mes='03'
Dia_act='26'

In [4]:
fecha_ini='2019-03-01 00:00:00'
fecha_fin='2019-03-31 21:00:00'

In [5]:
fecha_inivr='2019-03-01 00:00:00'
fecha_finivr='2019-03-31 21:00:00'

# CONSULTA DE GESTION MASIVOS ASESOR

In [6]:
querr_ultmes_asesor="""SELECT * 
                       FROM sistemecuador_atm.gestion 
                       where FECHA_GESTION between '{0}' and '{1}';""".format(fecha_ini,fecha_fin)

In [7]:
querr_ultmes_masivo="""SELECT * 
                       FROM sistemecuador_atm.gestion_ivr
                       where FECHA_GESTION between '{0}' and '{1}';""".format(fecha_inivr,fecha_finivr)

In [8]:
ultmes_asesor=pd.read_sql(querr_ultmes_asesor, con=db)

In [9]:
ultmes_asesor.columns

Index(['idGESTION', 'TIPO_GESTION', 'NRO_IDENTIFICACION_CLIENTE',
       'CASA_COBRANZA', 'COD_AGENTE', 'USUARIO_SAC', 'FECHA_GESTION',
       'ACCION_REALIZADA', 'RESPUESTA_OBTENIDA', 'CONTACTO_GESTIONO',
       'COMENTARIOS_GESTIONO', 'NUMERO_GESTION', 'MOTIVO_NO_PAGO',
       'FECHA_PROXIMA_GESTION', 'TIEMPO_GESTION', 'NRO_CUENTA_PROMESA',
       'VALOR_PROMESA', 'FECHA_PAGO_PROMESA', 'DIRECCION_VISITA', 'SUBCAMPAÑA',
       'CANAL_GESTION', 'archivo_audio'],
      dtype='object')

In [10]:
#ultmes_asesor[ultmes_asesor['NRO_IDENTIFICACION_CLIENTE']=='1000678738'][['NUMERO_GESTION','FECHA_GESTION','RESPUESTA_OBTENIDA']]

In [11]:
ultmes_masivo=pd.read_sql(querr_ultmes_masivo, con=db)

### ORDEN POR RESPUESTAS

In [12]:
ultmes_asesor.RESPUESTA_OBTENIDA = ultmes_asesor.RESPUESTA_OBTENIDA.astype('category')
order_resp_ase=['ACUERDO DE PAGO','ACUERDO DE CONVENIO','LOCALIZADO SIN ACUERDO',
                'YA PAGO','VOLVER A LLAMAR','INFRACTOR CUELGA LLAMADA','RECLAMO VENDIO VEHICULO',
                'RENUENTE','ACTUALIZACION DATOS','RECLAMO','MENSAJE CON TERCERO','FALLECIDO / OTROS',
                'AQUI NO VIVE / TRABAJA','BUZON DE MENSAJES','TELEFONO DANADO','TELEFONO OCUPADO','NO CONTESTA','','SIN GESTION']
ultmes_asesor.RESPUESTA_OBTENIDA=ultmes_asesor.RESPUESTA_OBTENIDA.cat.set_categories(order_resp_ase)

ultmes_asesor.CONTACTO_GESTIONO  = ultmes_asesor.CONTACTO_GESTIONO.astype('category')
order_cont_ase=['TITULAR','CLIENTE','TERC. VALIDO','TERC. NO VALIDO','NO CONTACTO','SIN GESTION']
ultmes_asesor.CONTACTO_GESTIONO=ultmes_asesor.CONTACTO_GESTIONO.cat.set_categories(order_cont_ase)

ultmes_asesor=ultmes_asesor.sort_values(by=['CONTACTO_GESTIONO','RESPUESTA_OBTENIDA'],ascending = True)

In [13]:
gest_ase_porID = ultmes_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'], keep='first')

In [14]:
gest_ase_porID_DEM = ultmes_asesor.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE','NUMERO_GESTION'], keep='first')

In [15]:
ultmes_masivo.RESPUESTA_OBTENIDA=ultmes_masivo.RESPUESTA_OBTENIDA.astype('category')
order_resp_ivr=['CONTESTA IVR','SMS ENVIADO','NO CONTESTA IVR','','SIN GESTION']
ultmes_masivo.RESPUESTA_OBTENIDA=ultmes_masivo.RESPUESTA_OBTENIDA.cat.set_categories(order_resp_ivr)

ultmes_masivo = ultmes_masivo.sort_values(by=['RESPUESTA_OBTENIDA'],ascending=True)

In [16]:
gest_mas_porID = ultmes_masivo.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE'],keep='first')

In [17]:
gest_mas_porID_DEM = ultmes_masivo.drop_duplicates(['NRO_IDENTIFICACION_CLIENTE','NUMERO_GESTION'],keep='first')

In [18]:
campos=['NRO_IDENTIFICACION_CLIENTE','RESPUESTA_OBTENIDA','CONTACTO_GESTIONO']
campos1=['NRO_IDENTIFICACION_CLIENTE','RESPUESTA_OBTENIDA','CONTACTO_GESTIONO','NUMERO_GESTION']

In [19]:
gest_ase_porID = gest_ase_porID[campos]
gest_ase_porID = gest_ase_porID.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MES','CONTACTO_GESTIONO':'CONT_MES'})

gest_ase_porID_DEM = gest_ase_porID_DEM[campos1]
gest_ase_porID_DEM = gest_ase_porID_DEM.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESTEL','CONTACTO_GESTIONO':'CONT_MESTEL','NUMERO_GESTION':'NUMERO'})

gest_mas_porID = gest_mas_porID[campos]
gest_mas_porID = gest_mas_porID.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESMAS','CONTACTO_GESTIONO':'CONT_MESMAS'})

gest_mas_porID_DEM = gest_mas_porID_DEM[campos1]
gest_mas_porID_DEM = gest_mas_porID_DEM.rename(columns={'RESPUESTA_OBTENIDA':'CALL_MESMASTEL','CONTACTO_GESTIONO':'CONT_MESMASTEL','NUMERO_GESTION':'NUMERO'})

In [20]:
gest_ase_porID_DEM.head()

,NRO_IDENTIFICACION_CLIENTE,CALL_MESTEL,CONT_MESTEL,NUMERO
204,0920338555,ACUERDO DE PAGO,TITULAR,0980969607
218,0921907549,ACUERDO DE PAGO,TITULAR,0992464217
219,0926784851,ACUERDO DE PAGO,TITULAR,0980790426
231,0915416713,ACUERDO DE PAGO,TITULAR,0986667147
253,0924619760,ACUERDO DE PAGO,TITULAR,0985719955


### ASIGNACION

In [21]:
def agregar_ceros(base):
    ced=base['IDENTIFICACION_DEUDOR']
    if len(ced)==9 or len(ced)==12 :
        if ced.isdigit():
            return '0'+ced
    return ced

In [22]:
def verificar_tipo(base):
    ced=base['ID']
    if len(ced)==13:
        return 'RUC'
    return 'CED'

In [23]:
def ultimo_digito(base):
    placa=base['PLACA_VEHICULO']
    placa=str(placa)
    if (placa == ''):
        return '10'
    if placa[-1].isdigit():
        return placa[-1]
    return '10'

In [24]:
def convenio(base):
    if (base['SALDO'] >= 115.8 and base['DIAS_MORA'] >= 180):
        return 'SI'
    return 'NO'

In [25]:
def diasmora(base):
    if(base['DIAS_MORAmin'] >= 180):
        return 'MAYOR A 180'
    return 'MENOR A 180'    

In [26]:
#nombre de los archivos Asignacion normal
nom_actual_cor='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_cor='Asignacion'
cartera_cor='CORRIENTE'

# nombre de los archivos Menores
nom_actual_men='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_men='Menores'
cartera_men='MENORES'

# nombre de los archivos Demografico
nom_actual_dem='..\..\..\..\BASES DATATOOLS\ASIGNACION\\2019\\ENERO\Sistema de cobro_'+Dia_act+'_'+Mes+'_'+Anio+'.xlsx'
nombre_hoja_dem='Demografico'

In [27]:
#leer archivo excel asignacion actual y anterior
asignacion_actual_cor=pd.read_excel(open(nom_actual_cor,'rb'), sheet_name=nombre_hoja_cor, dtype={"IDENTIFICACION_DEUDOR": str})
#leer archivo excel asignacion actual y anterior
# asignacion_actual_men=pd.read_excel(open(nom_actual_men,'rb'), sheet_name=nombre_hoja_men, dtype={"IDENTIFICACION_DEUDOR": str})
#leer archivo excel asignacion actual y anterior
asignacion_actual_dem=pd.read_excel(open(nom_actual_dem,'rb'), sheet_name=nombre_hoja_dem, dtype={"IDENTIFICACION_DEUDOR": str,'TELEFONO':str})

In [28]:
asignacion_actual_dem.head()

,IDENTIFICACION_DEUDOR,casa,TELEFONO,ESTADO_CREDITO,ESTADO,PRIORIDAD,TIPO
0,3778859,SISTEMA DE COBRO DEL ECUADOR,042457665,COACTIVO,BUENO,2.0,DOMICILIO
1,3778859,SISTEMA DE COBRO DEL ECUADOR,0980460504,COACTIVO,BUENO,2.0,DOMICILIO
2,5216855,SISTEMA DE COBRO DEL ECUADOR,042733103,COACTIVO,BUENO,2.0,DOMICILIO
3,5216855,SISTEMA DE COBRO DEL ECUADOR,0960503333,COACTIVO,BUENO,0.0,DOMICILIO
4,5216855,SISTEMA DE COBRO DEL ECUADOR,0999274687,COACTIVO,BUENO,NaN,DOMICILIO


In [29]:
asignacion_actual_cor= asignacion_actual_cor.fillna('')
asignacion_actual_cor['ID']=asignacion_actual_cor.apply(agregar_ceros,axis=1)

In [30]:
asignacion_actual_cor['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)

In [31]:
try:
    asignacion_actual_cor['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
except:
    try:
        asignacion_actual_cor['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
    except:
        asignacion_actual_cor['FECHA_COACTIVO']='SIN FECHA'

In [32]:
asignacion_actual_cor[asignacion_actual_cor['RAZONSOCIAL']==918217126]=''

In [33]:
asignacion_actual_cor['NOMBRE']=asignacion_actual_cor['RAZONSOCIAL']+' '+asignacion_actual_cor['PRIMER_APELLIDO']+' '+asignacion_actual_cor['SEGUNDO_APELLIDO']+' '+asignacion_actual_cor['PRIMER_NOMBRE']+' '+asignacion_actual_cor['SEGUNDO_NOMBRE']

In [34]:
asignacion_actual_cor['CARTERA']='CORRIENTE'

In [35]:
asignacion_actual_cor['TIPO']=asignacion_actual_cor.apply(verificar_tipo,axis=1)
asignacion_actual_cor['U_PLACA']=asignacion_actual_cor.apply(ultimo_digito,axis=1)

In [36]:
try:
    base_gestion_cor=asignacion_actual_cor[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
except:
    base_gestion_cor=asignacion_actual_cor[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
    ordenada_cor=base_gestion_cor.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [37]:
# asignacion_actual_men= asignacion_actual_men.fillna('')
# asignacion_actual_men['ID']=asignacion_actual_men.apply(agregar_ceros,axis=1)
# asignacion_actual_men['SUBCAMPANAPORCLIENTE'].replace(to_replace=[''],value='4. VENCIDA FASE I',inplace=True)
# try:
#     asignacion_actual_men['FECHA_COACTIVA'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
# except:
#     try:
#         asignacion_actual_men['FECHA_COACTIVO'].replace(to_replace=[''],value='NO COACTIVA',inplace=True)
#     except:
#         asignacion_actual_cor['FECHA_COACTIVO']='SIN FECHA'
# asignacion_actual_men['NOMBRE']=asignacion_actual_men['RAZONSOCIAL']+' '+asignacion_actual_men['PRIMER_APELLIDO']+' '+asignacion_actual_men['SEGUNDO_APELLIDO']+' '+asignacion_actual_men['PRIMER_NOMBRE']
# asignacion_actual_men['CARTERA']='MENORES'

In [38]:
# asignacion_actual_men['TIPO']=asignacion_actual_men.apply(verificar_tipo,axis=1)
# asignacion_actual_men['U_PLACA']=asignacion_actual_men.apply(ultimo_digito,axis=1)

In [39]:
# try:
#     base_gestion_men=asignacion_actual_men[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVA','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
#     ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVA':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})
# except:
#     base_gestion_men=asignacion_actual_men[['IDENTIFICACION_DEUDOR','ID','NOMBRE','SALDO','FECHA_COACTIVO','SUBCAMPANAPORCLIENTE','DIAS_MORA','CARTERA','TIPO','U_PLACA']]
#     ordenada_men=base_gestion_men.groupby(['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO']).agg({'FECHA_COACTIVO':'min','SALDO':'sum','DIAS_MORA':['min', 'max'],'U_PLACA':'min'})

In [40]:
# base_total=pd.concat([ordenada_cor, ordenada_men])
base_total=ordenada_cor

In [41]:
base_total = base_total.reset_index()

In [42]:
base_total.columns = ['IDENTIFICACION_DEUDOR','ID','NOMBRE','SUBCAMPANAPORCLIENTE','CARTERA','TIPO','FECHA_COACTIVA','SALDO','DIAS_MORAmin','DIAS_MORAmax','U_PLACA']

In [43]:
base_total.drop_duplicates(['IDENTIFICACION_DEUDOR'],keep='first',inplace=True)

In [44]:
base_total=base_total[base_total['IDENTIFICACION_DEUDOR']!='']

In [45]:
base_total['GENERACION_CONVENIO']='SI'
#base_total.apply(convenio,axis=1)
base_total['MORA']=base_total.apply(diasmora,axis=1)

In [46]:
base_total = base_total.rename(columns={'ID':'NRO_IDENTIFICACION_CLIENTE'})

In [47]:
base_total.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,U_PLACA,GENERACION_CONVENIO,MORA
0,00042448K,00042448K,ARAGON JUAREZ SERGIO OTTONIEL,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,155,155,9,SI,MENOR A 180
1,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,10,SI,MAYOR A 180
2,1000400828,1000400828,MORAN MADERA ANGEL MARCELO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,205,205,7,SI,MAYOR A 180
3,1000559797,1000559797,RECALDE ESPINOSA ALFONSO OSWALDO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,248.8,64,512,10,SI,MENOR A 180
4,1000584647,1000584647,DELGADO CRIBAN NORBERTO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,57.9,312,312,2,SI,MAYOR A 180


In [48]:
especiales = pd.read_excel(open('DEMOGRAFICOS.xlsx','rb'), sheet_name='BASE ESPECIAL', dtype={"IDENTIFICACION_DEUDOR": str})

In [49]:
base_total1 = pd.merge(base_total,especiales,on=['IDENTIFICACION_DEUDOR'],how='left')

### CRUCES CON LA ASIGNACION Y LAS GESTIONES

In [50]:
primercruce = pd.merge(base_total1,gest_ase_porID,on=['NRO_IDENTIFICACION_CLIENTE'],how='left')

In [51]:
segundocruce = pd.merge(primercruce,gest_mas_porID,on=['NRO_IDENTIFICACION_CLIENTE'],how='left')

In [52]:
segundocruce.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,U_PLACA,GENERACION_CONVENIO,MORA,CUOTA_VENCIDA,CALL_MES,CONT_MES,CALL_MESMAS,CONT_MESMAS
0,00042448K,00042448K,ARAGON JUAREZ SERGIO OTTONIEL,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,155,155,9,SI,MENOR A 180,SI,NaN,NaN,NaN,NaN
1,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,10,SI,MAYOR A 180,NaN,LOCALIZADO SIN ACUERDO,TITULAR,NaN,NaN
2,1000400828,1000400828,MORAN MADERA ANGEL MARCELO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,205,205,7,SI,MAYOR A 180,NaN,ACUERDO DE PAGO,TITULAR,NaN,NaN
3,1000559797,1000559797,RECALDE ESPINOSA ALFONSO OSWALDO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,248.8,64,512,10,SI,MENOR A 180,NaN,LOCALIZADO SIN ACUERDO,TITULAR,SMS ENVIADO,TITULAR
4,1000584647,1000584647,DELGADO CRIBAN NORBERTO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,57.9,312,312,2,SI,MAYOR A 180,NaN,NO CONTESTA,NO CONTACTO,NaN,NaN


In [53]:
segundocruce.NRO_IDENTIFICACION_CLIENTE.count()

34713

## Consultar numeros de la BD

### -Querry para la consulta de telefonos

In [54]:
consulta="""SELECT 
                CONCAT(CEDULA) as IDENTIFICACION_DEUDOR,
                CONCAT('0',NUMERO) as NUMERO,
                CASE CONTACTO
                    WHEN '1' THEN 1
                    WHEN '2' THEN 2
                    WHEN '3' THEN 5
                    WHEN '4' THEN 4
                    WHEN '5' THEN 3
                    WHEN ' ' THEN 3
                    WHEN '' THEN 3
                    WHEN 'TITULAR' THEN 1
                    WHEN 'TERCERO' THEN 2
                    WHEN 'EQUIVOCADO' THEN 5
                    WHEN 'NO CONTACTO' THEN 4
                    WHEN NULL THEN 3
                    ELSE  3
                END  AS  dueno
            FROM
                TELEFONICOS_NUMEROS.TELEFONOS  HAVING dueno IN (1,2,5)  ORDER BY dueno;"""

In [55]:
DEMO=pd.read_sql(consulta, con=db)

In [56]:
DEMO.IDENTIFICACION_DEUDOR.count()

281156

In [57]:
# DEMOS='DEMOGRAFICOS TOTALES.xlsx'
# writer = pd.ExcelWriter(DEMOS, engine='xlsxwriter')
# DEMO.to_excel(writer,sheet_name='DEMO')
# worksheet = writer.sheets['DEMO']
# writer.save()

In [58]:
DEMO.head()

,IDENTIFICACION_DEUDOR,NUMERO,dueno
0,,042000000,1
1,0001305593442,04275191,1
2,0917128589,0992221846,1
3,0C6J25T3L5,052585378,1
4,0G55941427,0991300718,1


In [59]:
leer_demografico=pd.read_excel(open('DEMOGRAFICOS.xlsx','rb'), sheet_name='DEMOGRAFICOS', dtype={"IDENTIFICACION_DEUDOR": str,"NUMERO":str,'DEMO':str})

In [60]:
leer_demografico.head()

,IDENTIFICACION_DEUDOR,DEMO,ESTADO,DEFINICION,NUMERO,NUEVO
0,425805,42426389,CORRECTO,TELEFONO,042426389,NO
1,102618907,824921,CORRECTO,TELEFONO,042824921,NO
2,102689486,72831423,CORRECTO,TELEFONO,072831423,NO
3,102969953,22572581,CORRECTO,TELEFONO,022572581,NO
4,201862992,42290540,CORRECTO,TELEFONO,042290540,NO


In [61]:
cruce_num = pd.merge(segundocruce,leer_demografico,on='IDENTIFICACION_DEUDOR',how='left')

In [62]:
cruce_num2 = pd.merge(cruce_num,DEMO,on=['IDENTIFICACION_DEUDOR','NUMERO'],how='left')
cruce_num2['dueno'].replace(to_replace=[1,2,5],value=['TITULAR','TERCERO','EQUIVOCADO'],inplace=True)

In [63]:
cruce_num2.drop_duplicates(['NUMERO','IDENTIFICACION_DEUDOR'], keep='first',inplace=True)

In [64]:
cruce_num2.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,...,CALL_MES,CONT_MES,CALL_MESMAS,CONT_MESMAS,DEMO,ESTADO,DEFINICION,NUMERO,NUEVO,dueno
0,00042448K,00042448K,ARAGON JUAREZ SERGIO OTTONIEL,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,155,155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,LOCALIZADO SIN ACUERDO,TITULAR,NaN,NaN,42018327,CORRECTO,TELEFONO,042018327,NO,TITULAR
2,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,LOCALIZADO SIN ACUERDO,TITULAR,NaN,NaN,990806079,CORRECTO,CELULAR,0990806079,NO,NaN
3,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,LOCALIZADO SIN ACUERDO,TITULAR,NaN,NaN,967093439,CORRECTO,CELULAR,0967093439,NO,NaN
4,1000400828,1000400828,MORAN MADERA ANGEL MARCELO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,205,205,...,ACUERDO DE PAGO,TITULAR,NaN,NaN,22239424,CORRECTO,TELEFONO,022239424,2019-03-08 00:00:00,TITULAR


In [65]:
cruce_uno = pd.merge(cruce_num2,gest_ase_porID_DEM,on=['NRO_IDENTIFICACION_CLIENTE','NUMERO'],how='left')
cruce_dos = pd.merge(cruce_uno,gest_mas_porID_DEM,on=['NRO_IDENTIFICACION_CLIENTE','NUMERO'],how='left')

In [66]:
cruce_dos.head()

,IDENTIFICACION_DEUDOR,NRO_IDENTIFICACION_CLIENTE,NOMBRE,SUBCAMPANAPORCLIENTE,CARTERA,TIPO,FECHA_COACTIVA,SALDO,DIAS_MORAmin,DIAS_MORAmax,...,DEMO,ESTADO,DEFINICION,NUMERO,NUEVO,dueno,CALL_MESTEL,CONT_MESTEL,CALL_MESMASTEL,CONT_MESMASTEL
0,00042448K,00042448K,ARAGON JUAREZ SERGIO OTTONIEL,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,155,155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,42018327,CORRECTO,TELEFONO,042018327,NO,TITULAR,LOCALIZADO SIN ACUERDO,TITULAR,NaN,NaN
2,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,990806079,CORRECTO,CELULAR,0990806079,NO,NaN,NaN,NaN,NaN,NaN
3,100025030,0100025030,CRESPO DELGADO JULIO TEODORO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,19.3,332,332,...,967093439,CORRECTO,CELULAR,0967093439,NO,NaN,NaN,NaN,NaN,NaN
4,1000400828,1000400828,MORAN MADERA ANGEL MARCELO,MULTAS NUEVAS,CORRIENTE,CED,NO COACTIVA,115.8,205,205,...,22239424,CORRECTO,TELEFONO,022239424,2019-03-08 00:00:00,TITULAR,ACUERDO DE PAGO,TITULAR,NaN,NaN


In [67]:
asignacion_actual_dem['ASIGNADO']='SI'
asig_demo=asignacion_actual_dem[['IDENTIFICACION_DEUDOR','TELEFONO','ASIGNADO']]

In [68]:
asig_demo = asig_demo.rename(columns={'TELEFONO':'NUMERO'})

In [69]:
asig_demo.head()

,IDENTIFICACION_DEUDOR,NUMERO,ASIGNADO
0,3778859,042457665,SI
1,3778859,0980460504,SI
2,5216855,042733103,SI
3,5216855,0960503333,SI
4,5216855,0999274687,SI


In [70]:
verificacion = pd.merge(cruce_dos,asig_demo,on=['IDENTIFICACION_DEUDOR','NUMERO'],how='left')

In [71]:
verificacion.drop_duplicates(['IDENTIFICACION_DEUDOR','NUMERO'], keep='first',inplace=True)

In [72]:
verificacion.columns

Index(['IDENTIFICACION_DEUDOR', 'NRO_IDENTIFICACION_CLIENTE', 'NOMBRE',
       'SUBCAMPANAPORCLIENTE', 'CARTERA', 'TIPO', 'FECHA_COACTIVA', 'SALDO',
       'DIAS_MORAmin', 'DIAS_MORAmax', 'U_PLACA', 'GENERACION_CONVENIO',
       'MORA', 'CUOTA_VENCIDA', 'CALL_MES', 'CONT_MES', 'CALL_MESMAS',
       'CONT_MESMAS', 'DEMO', 'ESTADO', 'DEFINICION', 'NUMERO', 'NUEVO',
       'dueno', 'CALL_MESTEL', 'CONT_MESTEL', 'CALL_MESMASTEL',
       'CONT_MESMASTEL', 'ASIGNADO'],
      dtype='object')

In [73]:
verificacion = verificacion.rename(columns={'NRO_IDENTIFICACION_CLIENTE':'CED'})

In [74]:
def order(frame,var):
    if type(var) is str:
        var = [var] #let the command take a string or list
    varlist =[w for w in frame.columns if w not in var]
    frame = frame[var+varlist]
    return frame

In [75]:
ordenColumn = ['NUMERO','NOMBRE','CED','SALDO','SUBCAMPANAPORCLIENTE','FECHA_COACTIVA','CARTERA','TIPO','DIAS_MORAmin','DIAS_MORAmax','U_PLACA']

In [76]:
verificacion1 = order(verificacion,ordenColumn)

In [77]:
nuevo_archivo='BASE ARREGLADA MARZO.xlsx'
writer = pd.ExcelWriter(nuevo_archivo, engine='xlsxwriter')
verificacion1.to_excel(writer,sheet_name='BASE_TOTAL')
worksheet = writer.sheets['BASE_TOTAL']
writer.save()